In [1]:
%run ../lib.ipynb
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../..")
from analysis_tools import KmerAligner

import pickle

In [2]:
path = "../../../../NanoporeData/burnin_downloads/"
SRATE = 5000
NMERS = 5

In [3]:
files = !find {path} | grep fast5

In [4]:
ref = !cat ../../../../NanoporeData/LambdaReference/lambda_ref.fasta | grep -v ">"
ref = ref[0]

In [5]:
file_data = process_metrichor_files(files, verbose=True)

processing file 0 channel 101
processing file 11 channel 101
processing file 12 channel 101
processing file 14 channel 101
processing file 15 channel 101
processing file 16 channel 101
processing file 17 channel 101
processing file 18 channel 101
processing file 19 channel 101
processing file 1 channel 101
processing file 21 channel 101
processing file 22 channel 101
processing file 23 channel 101
processing file 25 channel 101
processing file 26 channel 101
processing file 2 channel 101
processing file 3 channel 101
processing file 4 channel 101
processing file 5 channel 101
Empty file: ../../../../NanoporeData/burnin_downloads/IBIS218_010_059_ibisburnin_1700_1_ch101_file5_strand.fast5. Skipped.
processing file 6 channel 101
processing file 7 channel 101
processing file 8 channel 101
processing file 9 channel 101
processing file 0 channel 102
processing file 1 channel 102
processing file 2 channel 102
processing file 3 channel 102
processing file 5 channel 102
Empty file: ../../../../

OSError: [Errno 12] Cannot allocate memory

In [ ]:
pickle.dump(file_data, open("file_data_phage.pickle", 'wb'))

In [41]:
NMER = 3
kmer_aligner = KmerAligner(ref, verbose = True, nmer=NMER)

Query Length 48502


In [42]:
true_events = []

for file in file_data:
    tmp_events = kmer_aligner.align_file(file)
    true_events.extend(tmp_events)
    
kmer_aligner.print_statistics()

processing ch 104 file 18

        Start/End Query 26890/32703 Target 2/5637 Skipped 5
        Length 5635/5640 Identity 0.25750
        
processing ch 104 file 19

        Start/End Query 11578/20191 Target 0/7179 Skipped 2
        Length 7179/7181 Identity 0.31509
        
processing ch 104 file 1

        Start/End Query 2998/3133 Target 0/131 Skipped 2
        Length 131/133 Identity 0.49618
        
processing ch 104 file 3

        Start/End Query 14580/20130 Target 1/5074 Skipped 2
        Length 5073/5075 Identity 0.28070
        
processing ch 104 file 4

        Start/End Query 43021/47843 Target 1/4575 Skipped 3
        Length 4574/4577 Identity 0.25120
        
processing ch 104 file 6

        Start/End Query 23494/32229 Target 1/8087 Skipped 4
        Length 8086/8090 Identity 0.28135
        
processing ch 104 file 7

        Start/End Query 36798/38473 Target 1/1523 Skipped 2
        Length 1522/1524 Identity 0.36925
        
processing ch 104 file 8

        Start/End 

In [45]:
import pysam

In [46]:
samfile = pysam.AlignmentFile("../../NanoporeData/burnin_downloads.sorted.bam")

In [56]:
reads = [x for x in samfile.fetch()]

In [86]:
reads[200].get_aligned_pairs()
read = reads[4000]

In [88]:
# print(read.get_aligned_pairs())
read.qname

'ch443_file3_twodirections:burnin_downloads/IBIS218_010_059_ibisburnin_1700_1_ch443_file3_strand.fast5'

### create bam from sequences

In [89]:
with open("phage_all_seqs.fasta", 'w') as f: 
    for file_obj in file_data:
        f.write(">ch{0}_file{1}_template\n".format(file_obj["channel"], file_obj["file_id"]))
        f.write(file_obj["called_seq"] + "\n")

In [97]:
!bowtie2-build ../../NanoporeData/LambdaReference/lambda_ref.fasta "lambda-index"

Settings:
  Output files: "lambda-index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ../../NanoporeData/LambdaReference/lambda_ref.fasta
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 12125
Using parameters --bmax 9094 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9094 --dcv 1024
Constructing suffi

In [108]:
path = !pwd
path = path[0]
!bowtie2 --local -x {path}/lambda-index -S test.sam -f {path}/phage_all_seqs.fasta 

58 reads; of these:
  58 (100.00%) were unpaired; of these:
    52 (89.66%) aligned 0 times
    2 (3.45%) aligned exactly 1 time
    4 (6.90%) aligned >1 times
10.34% overall alignment rate
